In [2]:
# Some tests of the various GOES flare lists
# 
# 12-Apr-2023   IGH

In [10]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy import timeseries as ts
from sunpy.time import parse_time
from astropy.io import fits
import astropy.units as u
import datetime
import pandas as pd
import os
from pathlib import Path
import glob
from stixdcpy.quicklook import LightCurves

import nsx_func

import warnings
warnings.simplefilter('ignore')

plt.rcParams.update({'font.size': 18,'font.family':"sans-serif",\
            'font.sans-serif':"Arial",'mathtext.default':"regular"})

In [4]:
# List of all the NuSTAR observations, with my naming, 
# based off first day of observing per campaign
dobs=['20140910','20141101','20141211',
    '20150429','20150901',
    '20160219','20160422','20160726',
    '20170321','20170821','20170911','20171010',
    '20180529','20180907','20180928',
    '20190112','20190412','20190425','20190702',
    '20200129','20200221','20200606','20200912',
    '20210108','20210429','20210720','20210730','20211117',
    '20220224','20220603','20220906','20221209',
    '20230318']

# Data directory
ddir=str(Path.home())+'/data/heasarc_nustar/'

In [11]:
# Select a pointing with some flares during it 
# (not necesarily where NuSTAR is looking though)

icamp=32
maindir=ddir+'ns_'+dobs[icamp]+'/'
print(maindir)
# Most start with 20 or 90?
ids = [f.name for f in os.scandir(maindir) \
    if (f.is_dir() and (f.name.startswith('20') or f.name.startswith('90')))]
ids=sorted(ids)
nsid=ids[0]

df_test=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=False,englow=2,enghigh=10)
df10 = df_test.resample('10s', level=0).mean()

# Time range to nearest 5mins
mint=df10.index[0].to_pydatetime()
mint-=datetime.timedelta(minutes=5,seconds=mint.second)
mint-=datetime.timedelta(minutes=mint.minute % 5)
maxt=df10.index[-1].to_pydatetime()
maxt+=datetime.timedelta(minutes=5,seconds=60-maxt.second)
maxt-=datetime.timedelta(minutes=maxt.minute % 5)
tr=[mint,maxt]
trange=a.Time(tr[0],tr[1])



/Users/iain/data/heasarc_nustar/ns_20230318/


In [18]:
# gsflin = Fido.search(trange,a.hek.EventType("FL"),a.hek.FRM.Name == "SSW Latest Events")
# # gsflhk=gsflin["hek"]
# gsflin

# gsflin2 = Fido.search(trange,a.hek.EventType("FL"),a.hek.FRM.Name == "SWPC")
gsflin2 = Fido.search(trange,a.hek.EventType("FL"),a.hek.OBS.Observatory == "GOES")
print(gsflin2)

Results from 1 Provider:

0 Results from the HEKClient:

<No columns>




In [28]:
print(trange.start.isot)

2023-03-18T13:25:00.000


In [50]:
from stixdcpy.net import Request as jreq

stxfl=jreq.fetch_flare_list(trange.start.isot, trange.end.isot, sort='time')
stxfl.dataframe()

,flare_id,duration,att_in,peak_UTC,LC0_BKG,_id,GOES_class,GOES_flux,CFL_X_arcsec,CFL_Y_arcsec,...,LC0_PEAK_COUNTS_4S,LC0_BKG_COUNTS_4S,LC1_PEAK_COUNTS_4S,LC1_BKG_COUNTS_4S,LC2_PEAK_COUNTS_4S,LC2_BKG_COUNTS_4S,LC3_PEAK_COUNTS_4S,LC3_BKG_COUNTS_4S,LC4_PEAK_COUNTS_4S,LC4_BKG_COUNTS_4S
0,2303181421,752,False,2023-03-18T14:21:08.848,219.988274,25484,C1.1,1.066117e-06,None,None,...,607,287.0,99,45.0,83,61.0,799,735.0,431,383.0
1,2303181358,108,False,2023-03-18T13:58:48.845,219.988274,25483,B8.4,8.350112e-07,None,None,...,303,287.0,49,45.0,67,61.0,799,735.0,431,383.0
2,2303181349,428,False,2023-03-18T13:49:24.844,219.988274,25482,B8.4,8.408853e-07,None,None,...,335,287.0,67,45.0,75,61.0,799,735.0,431,383.0
3,2303181334,760,False,2023-03-18T13:34:08.843,219.988274,25481,C1.1,1.128526e-06,None,None,...,2431,287.0,431,45.0,247,61.0,799,735.0,431,383.0


In [52]:
# Could use STIX flare list instead but no flare positions just the times
for sf in stxfl:
    print(sf.get('peak_UTC'),sf.get('start_UTC'),sf.get('end_UTC'))
# Need to shift these to the times at the Earth?

2023-03-18T14:21:08.848 2023-03-18T14:17:44.847 2023-03-18T14:30:16.849
2023-03-18T13:58:48.845 2023-03-18T13:58:08.845 2023-03-18T13:59:56.845
2023-03-18T13:49:24.844 2023-03-18T13:48:28.844 2023-03-18T13:55:36.845
2023-03-18T13:34:08.843 2023-03-18T13:27:16.842 2023-03-18T13:39:56.843
